Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give read access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-11 13:32:28--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-11 13:32:29--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K   115KB/s    in 1.8s    

2021-10-11 13:32:31 (115 KB/s) - ‘data.zip’ saved [217313/217313]



In [2]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   creating: data/validation/
  inflating: data/validation/validation.data.txt  
  inflating: data/validation/validation.gold.txt  


In [3]:
## Import relevant packages
import os
import re
import numpy as np
import json
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Field, TabularDataset, BucketIterator
from torchtext.vocab import GloVe

In [4]:
## Limiting randomness
torch.manual_seed(1)
np.random.seed(0)

## Creating .CSV Dataset

In [5]:
def csv_creator(path_data='/content/data/train/train.data.txt', path_gold='/content/data/train/train.gold.txt', path_dir='content/', is_label=False, kind='train'):
    ## Reading Data File
    train_list = []
    count = 1
    with open(path_data, 'r') as f:
        for line in f:
            line = line.rstrip('\n')
            line = line.split('\t')
            token = line[0]
            entity = line[1]
            pos = line[2].split('-')
            pos1 = pos[0]
            pos2 = pos[1]
            sent1 = line[3]
            ## Punctuation if removed shall change the current order of words 
            ## and thus the significance of the indices given shall be lost
            sent1 = ' '.join(sent1.split())
            sent2 = line[4]
            sent2 = ' '.join(sent2.split())
            train_list.append({"Token": token, "Entity": entity, "Position1": pos1, 
                              "Position2": pos2, "Sentence1": sent1, "Sentence2": sent2})
        f.close()

    ## Creating DataFrame for the data
    train_df = pd.DataFrame(train_list)

    ## Label Encoding
    train_df['Entity'] = np.where(train_df['Entity'] == 'V', 1, 0)

    # Reading Gold Labels
    if is_label:
        label_list = []
        with open(path_gold, 'r') as f:
            label_list = f.read().splitlines()
            f.close()

        ## Label Encoding
        for i in range(len(label_list)):
            if label_list[i] == 'T':
                label_list[i] = 1
            else:
                label_list[i] = 0
        train_df['Label'] = label_list
    
    ## Saving Data File
    path_csv = path_dir + str(kind) + '.csv'
    train_df.to_csv(path_csv)

In [6]:
## Creating dataset for Train File
csv_creator(path_data='/content/data/train/train.data.txt', path_gold='/content/data/train/train.gold.txt', path_dir = '/content/', is_label=True, kind='train')

In [7]:
## Creating dataset for Validation File
csv_creator(path_data='/content/data/validation/validation.data.txt', path_gold='/content/data/validation/validation.gold.txt', path_dir = '/content/', is_label=True, kind='validation')

## Converting Dataset to Iteratable Format

In [8]:
## Checking available device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [9]:
tokenizer = lambda x: x.split()

In [10]:
## Creating two necessary fields to encode the structure of the datasets
text_field = Field(sequential=True, use_vocab=True, tokenize=tokenizer, lower=True, batch_first=True, include_lengths=True)
label_field = Field(sequential=False, use_vocab=False, batch_first=True)

In [11]:
## Defining Fields for Training and Validation Data
fields = {'Token': ('t', text_field), 'Entity': ('e', label_field), 'Position1': ('p1', label_field), 
          'Position2': ('p2', label_field), 'Sentence1': ('s1', text_field), 'Sentence2': ('s2', text_field), 'Label': ('l', label_field)}

In [12]:
## Converting dataset into torch readable format using TorchText
train_data, test_data = TabularDataset.splits(
    path = '/content/',
    train = 'train.csv',
    test = 'validation.csv',
    format = 'csv',
    fields = fields,
)

In [13]:
print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())

dict_keys(['t', 'e', 'p1', 'p2', 's1', 's2', 'l'])
dict_values([['carry'], '1', '2', '1', ['you', 'must', 'carry', 'your', 'camping', 'gear', '.'], ['sound', 'carries', 'well', 'over', 'water', '.'], '0'])


In [14]:
# Building Vocabulary
text_field.build_vocab(train_data, min_freq=1, vectors='glove.6B.300d')

.vector_cache/glove.6B.zip: 862MB [02:39, 5.39MB/s]                           
100%|█████████▉| 399999/400000 [00:45<00:00, 8750.03it/s]


In [16]:
len(text_field.vocab)

7460

In [17]:
# print(list(text_field.vocab.stoi.items()))

In [18]:
## Splitting data into batches
train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size=32,
    sort_key=lambda x: len(list(x.s1)),
    device=device,
    sort_within_batch=True,
    shuffle=False         ## To ensure no shuffling of data is there
)

In [19]:
## Checking shuffling (if there)
# count = 0
# for batch_no, batch in enumerate(train_iterator):
#   if count == 0:
#     text, batch_len = batch.s1
#     print(text, batch_len)
#     print(batch.l)
#     count += 1

In [20]:
## Checking batches
# count = 0
# for batch in train_iterator:
#     if count < 5:
#         print(batch)
#         count += 1

## Defining Model

In [21]:
class BiLSTM_WordMeaningComparison(nn.Module):
    def __init__(self, vocab_size, embed_dim,
                 hidden, n_layers):
        super(BiLSTM_WordMeaningComparison, self).__init__()    ## loading from parent class
        self.hidden = hidden
        self.n_layers = n_layers
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.embed.weight.data.copy_(text_field.vocab.vectors)
        self.embed.weight.requires_grad = False     ## Freezing the embeddings
        self.lstm = nn.LSTM(embed_dim, hidden,
                            num_layers=n_layers,
                            bidirectional=True,     ## makes it a BiLSTM
                            batch_first=True)
        drp1 = 0.5      # dropout probability
        self.dropout1 = nn.Dropout(drp1)
        self.fc1 = nn.Linear(hidden * 2, 32)        ## fully connected layer

    def forward(self, input, actual_batch_len):
        embed_out = self.embed(input)
        hidden = torch.zeros(self.n_layers * 2, input.shape[0], self.hidden, requires_grad=True).to(device)   ## hidden state
        cell = torch.zeros( self.n_layers * 2, input.shape[0], self.hidden, requires_grad=True).to(device)    ## cell state
        out_lstm, (hidden, cell) = self.lstm(embed_out,(hidden, cell)) # Applying BiLSTM Model
        out_lstm = self.dropout1(out_lstm)                             # Applying dropout
        out_lstm = self.fc1(out_lstm)                                  # Applying Fully connected layer
        return out_lstm

## Defining input parameters for the model
vocab_dim = len(text_field.vocab)
embedding_dim = text_field.vocab.vectors.shape[1]
hidden_dim = 256
num_layers = 1
model = BiLSTM_WordMeaningComparison(vocab_dim, embedding_dim, hidden_dim, num_layers)

In [22]:
## Defining Optimizer and Criterion
opt = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CosineEmbeddingLoss()          ## to gain a measure of similarity between the two word embeddings
model.to(device)

BiLSTM_WordMeaningComparison(
  (embed): Embedding(7460, 300)
  (lstm): LSTM(300, 256, batch_first=True, bidirectional=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=512, out_features=32, bias=True)
)

## Training Model

In [23]:
## Defining accuracy measurements
def accuracy(preds, y):
    acc = torch.sum(preds == y) / y.shape[0]
    return acc

## Function to Calculate Loss
def calculateLoss(model, batch, criterion):
    ## Running BiLSTM for Sentence 1
    text, text_len = batch.s1
    p1 = batch.p1
    embeddings = model(text, text_len.to(device))

    ## Running BiLSTM for Sentence 2
    text_2, text_len_2 = batch.s2
    p2 = batch.p2
    embeddings_2 = model(text_2, text_len_2.to(device))

    ## Picking out the specific word embeddings from the output for sentence 1
    j = 1
    first = embeddings[0, p1[0], :]
    for i in p1[1:]:
        first = torch.vstack((first, embeddings[j, i, :]))
        j += 1

    ## Picking out the specific word embeddings from the output for sentence 2
    j = 1
    second = embeddings_2[0, p2[0], :]
    for i in p2[1:]:
        second = torch.vstack((second, embeddings_2[j, i, :]))
        j += 1

    ## Creating similarity and dissimilarity vector for segregating results
    pred = batch.l.float()
    y = pred.clone().detach()       ## creating copy of the batch labels
    for i in range(len(y)):
      if y[i] == 0:
          y[i] = -1

    ## Calculating Loss
    loss = criterion(first, second, y)
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    res = cos(first, second).float()

    for i in range(len(res)):
        if res[i] > 0.75:      
          res[i] = 1
        else:
          res[i] = 0

    ## Generating Accuracy
    acc = accuracy(res, pred)
    return loss, batch.l.shape[0], acc

In [24]:
## Checkpointing: Storing model state
def save_ckp(path, epoch, model, opt, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': loss,
            }, path)

In [25]:
epochs_total = 20
val_loss_min = 10
for epoch in range(epochs_total):
    model.train()       ## Training the model
    train_len, train_acc, train_loss  = 0, [], []
    for batch_no, batch in enumerate(train_iterator):
        opt.zero_grad()
        loss, batch_len, acc = calculateLoss(model, batch, criterion)
        train_loss.append(loss * batch_len)
        train_acc.append(acc * batch_len)
        train_len = train_len + batch_len
        loss.backward()
        opt.step()

    ## Calculating the loss and accuracy terms for each epoch
    train_epoch_loss = np.sum(train_loss)/train_len
    train_epoch_acc = np.sum(train_acc)/train_len
    model.eval()

    ## Finding out the validation results; backward step not needed here since this is not be trained on
    with torch.no_grad():
        for batch in test_iterator:
            val_results = [calculateLoss(model, batch,criterion) for batch in test_iterator]
            loss, batch_len, valid_acc = zip(*val_results)
            epoch_loss = np.sum(np.multiply(loss, batch_len))/np.sum(batch_len)
            epoch_acc  = np.sum(np.multiply(valid_acc , batch_len))/np.sum(batch_len)
        print('Epoch Number:{}/{} | Training Loss: {:.4f}, Training Accuracy: {:.4f}'
              ' Validation Loss: {:.4f}, Validation Accuracy: {:.4f}'
              .format(epoch+1, epochs_total, train_epoch_loss.item(), 
                      train_epoch_acc.item(), epoch_loss.item(), epoch_acc.item()))

        ## Saving the model for least val_loss
        val_loss = epoch_loss.item()
        val_acc = epoch_acc.item()
        if val_loss < val_loss_min:
            val_loss_min = val_loss
            print('Validation Loss Decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_loss_min,val_loss))
            ## Saving checkpoint
            save_ckp('model.txt', epoch+1, model, opt, val_loss_min)
            

Epoch Number:1/20 | Training Loss: 0.4506, Training Accuracy: 0.5621 Validation Loss: 0.4773, Validation Accuracy: 0.5313
Validation Loss Decreased (0.477338 --> 0.477338).  Saving model ...
Epoch Number:2/20 | Training Loss: 0.4108, Training Accuracy: 0.6275 Validation Loss: 0.4719, Validation Accuracy: 0.5564
Validation Loss Decreased (0.471850 --> 0.471850).  Saving model ...
Epoch Number:3/20 | Training Loss: 0.3636, Training Accuracy: 0.6630 Validation Loss: 0.4576, Validation Accuracy: 0.5627
Validation Loss Decreased (0.457590 --> 0.457590).  Saving model ...
Epoch Number:4/20 | Training Loss: 0.3321, Training Accuracy: 0.6995 Validation Loss: 0.4527, Validation Accuracy: 0.5517
Validation Loss Decreased (0.452698 --> 0.452698).  Saving model ...
Epoch Number:5/20 | Training Loss: 0.3017, Training Accuracy: 0.7229 Validation Loss: 0.4423, Validation Accuracy: 0.5752
Validation Loss Decreased (0.442340 --> 0.442340).  Saving model ...
Epoch Number:6/20 | Training Loss: 0.2698, Tr

In [27]:
vocab = text_field.vocab
torch.save(vocab, 'vocab.txt')

In [28]:
## Zip the final model and all the required files, such as vocabulary
# Replace USERID with your own, such as 2017CSZ8058
!zip -r 2018ME10032_A_model.zip model.txt vocab.txt

## Upload it to Google drive and ensure that the testing notebook uses the correct link

  adding: model.txt (deflated 9%)
  adding: vocab.txt (deflated 11%)
